[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/PetiteIA/schema_mechanism/blob/master/notebooks/agent7.ipynb)

# THE AGENT WHO PROGRAMMED ITSELF

# Learning objectives

Upon completing this lab, you will be able to implement a developmental agent that can re-enact a whole sequence of interaction.

# Define the necessary classes

Ensure the required packages are installed if they aren't already.

In [ ]:
!pip install pandas
!pip install numpy
!pip install matplotlib
!pip install ipywidgets
!pip install IPython.display

We keep improving the Interaction class.

In [113]:
class Interaction:
    """An interaction is a tuple (action, outcome) with a valence"""
    def __init__(self, action, outcome, valence):
        self._action = action
        self._outcome = outcome
        self._valence = valence

    def get_action(self):
        """Return the action"""
        return self._action

    def get_outcome(self):
        """Return the action"""
        return self._outcome

    def get_valence(self):
        """Return the action"""
        return self._valence

    def key(self):
        """ The key to find this interaction in the dictinary is the string '<action><outcome>'. """
        return f"{self._action}{self._outcome}"

    def pre_key(self):
        """Return the key. Used for compatibility with CompositeInteraction"""
        return self.key()

    def __str__(self):
        """ Print interaction in the form '<action><outcome:<valence>' for debug."""
        return f"{self._action}{self._outcome}:{self._valence}"

    def __eq__(self, other):
        """ Interactions are equal if they have the same key """
        if isinstance(other, self.__class__):
            return self.key() == other.key()
        else:
            return False            

We keep improving the CompositeInteraction class.

In [114]:
class CompositeInteraction:
    """A composite interaction is a tuple (pre_interaction, post_interaction) and a weight"""
    def __init__(self, pre_interaction, post_interaction):
        self.pre_interaction = pre_interaction
        self.post_interaction = post_interaction
        self.weight = 1
        self.isActivated = False

    def get_action(self):
        """Return the action of the pre interaction"""
        return self.key()
        return self.pre_interaction.get_action()
    
    def get_valence(self):
        """Return the valence of the pre_interaction plus the valence of the post_interaction"""
        return self.pre_interaction.get_valence() + self.post_interaction.get_valence()

    def reinforce(self):
        """Increment the composite interaction's weight"""
        self.weight += 1

    def key(self):
        """ The key to find this interaction in the dictionary is the string '<pre_interaction><post_interaction>'. """
        return f"({self.pre_interaction.key()},{self.post_interaction.key()})"

    def pre_key(self):
        """Return the key of the pre_interaction"""
        return self.pre_interaction.pre_key()

    def __str__(self):
        """ Print the interaction in the Newick tree format (pre_interaction, post_interaction: valence) """
        return f"({self.pre_interaction}, {self.post_interaction}: {self.weight})"

    def __eq__(self, other):
        """ Interactions are equal if they have the same pre and post interactions """
        if isinstance(other, self.__class__):
            return (self.pre_interaction == other.pre_interaction) and (self.post_interaction == other.post_interaction)
        else:
            return False

## Define the Agent class

Let's implement a base Agent that has the functionnalities of Agent6.

In [198]:
import pandas as pd

class Agent:
    def __init__(self, _interactions):
        """ Initialize our agent """
        self._interactions = {interaction.key(): interaction for interaction in _interactions}
        self._composite_interactions = {}
        self._intended_interaction = self._interactions["00"]
        self._last_interaction = None
        self._previous_interaction = None
        self._penultimate_interaction = None
        self._last_composite_interaction = None
        self._previous_composite_interaction = None
        # Create a dataframe of default primitive interactions 
        default_interactions = [interaction for interaction in _interactions if interaction.get_outcome() == 0]
        data = {'interaction': [i.key() for i in default_interactions],
                'action': [i.get_action() for i in default_interactions],
                'weight': [0] * len(default_interactions),
                'proclivity': [0] * len(default_interactions)}
        self.primitive_df = pd.DataFrame(data)
        # Store the selection dataframe as a class attribute so we can display it in the notebook
        self.selection_df = None

    def action(self, _outcome):
        """Implement the agent's policy"""
        # tracing the previous cycle
        self._previous_composite_interaction = self._last_composite_interaction
        self._penultimate_interaction = self._previous_interaction
        self._previous_interaction = self._last_interaction
        self._last_interaction = self._interactions[f"{self._intended_interaction.get_action()}{_outcome}"]
        print(f"Action: {self._intended_interaction.get_action()}, Prediction: {self._intended_interaction.get_outcome()}, "
              f"Outcome: {_outcome}, Prediction_correct: {self._intended_interaction.get_outcome() == _outcome}, "
              f"Valence: {self._last_interaction.get_valence()}")

        # Call the learning mechanism
        self.learn()
        
        # Create a dataframe from the activated composite interaction 
        activated_keys = [composite_interaction.key() for composite_interaction in self._composite_interactions.values() 
                          if composite_interaction.pre_interaction == self._last_interaction or 
                          composite_interaction.pre_interaction == self._last_composite_interaction]
        data = {'composite': activated_keys,
                'weight': [self._composite_interactions[k].weight for k in activated_keys],
                'post_valence': [self._composite_interactions[k].post_interaction.get_valence() for k in activated_keys],
                'action': [self._composite_interactions[k].post_interaction.get_action() for k in activated_keys],
                'interaction': [self._composite_interactions[k].post_interaction.pre_key() for k in activated_keys]
                }
        activated_df = pd.DataFrame(data)

        # Create the selection dataframe from the primitive and the activated dataframes
        df = pd.concat([self.primitive_df, activated_df], ignore_index=True)

        # Compute the proclivity for each action
        df['proclivity'] = df['weight'] * df['post_valence']
        grouped_df = df.groupby('action').agg({'proclivity': 'sum'}).reset_index()
        df = df.merge(grouped_df, on='action', suffixes=('', '_sum'))

        # Find the most probable outcome for each action
        max_weight_df = df.loc[df.groupby('action')['weight'].idxmax(), ['action', 'interaction']].reset_index(drop=True)
        max_weight_df.columns = ['action', 'intended']
        df = df.merge(max_weight_df, on='action')

        # Find the first row that has the highest proclivity
        max_index = df['proclivity_sum'].idxmax()
        intended_interaction_key = df.loc[max_index, ['intended']].values[0]
        self._intended_interaction = self._interactions[intended_interaction_key]
        print("Intended", self._intended_interaction)

        # Store the selection dataframe for printing
        self.selection_df = df.copy()
        
        return self._intended_interaction.get_action()

    def learn(self):
        """Learn the composite interactions"""
        # First level of composite interactions
        self._last_composite_interaction = self.learn_composite_interaction(self._previous_interaction, self._last_interaction)
        # Second level of composite interactions
        self.learn_composite_interaction(self._previous_composite_interaction, self._last_interaction)
        self.learn_composite_interaction(self._penultimate_interaction, self._last_composite_interaction)
    
    def learn_composite_interaction(self, pre_interaction, post_interaction):
        if pre_interaction is None:
            return None
        else:
            # Record or reinforce the first level composite interaction
            composite_interaction = CompositeInteraction(pre_interaction, post_interaction)
            if composite_interaction.key() not in self._composite_interactions:
                self._composite_interactions[composite_interaction.key()] = composite_interaction
                print(f"Learning {composite_interaction}")
                return composite_interaction
            else:
                self._composite_interactions[composite_interaction.key()].reinforce()
                print(f"Reinforcing {self._composite_interactions[composite_interaction.key()]}")
                # Retrieve the existing composite interaction
                return self._composite_interactions[composite_interaction.key()]

# PRELIMINARY EXERCISE

## Create Environment6

The agent has two possible actions: move to the left or move to the right. 
The environment returns outcome 1 when the agent bumps into a light green wall, and then the wall turns dark green until the agent moves away.

In [199]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import Output
from IPython.display import display

class Environment6:
    """ The grid """
    def __init__(self):
        """ Initialize the grid """
        self.grid = np.array([[1, 0, 0, 1]])
        self.position = 1

    def outcome(self, action):
        """Take the action and generate the next outcome """
        if action == 0:
            # Move left
            if self.position > 1:
                # No bump
                self.position -= 1
                self.grid[0, 3] = 1
                outcome = 0
            elif self.grid[0, 0] == 1:
                # First bump
                outcome = 1
                self.grid[0, 0] = 2
            else:
                # Subsequent bumps
                outcome = 0
        else:
            # Move right
            if self.position < 2:
                # No bump
                self.position += 1
                self.grid[0, 0] = 1
                outcome = 0
            elif self.grid[0, 3] == 1:
                # First bump
                outcome = 1
                self.grid[0, 3] = 2
            else:
                # Subsequent bumps
                outcome = 0
        return outcome  

    def display(self):
        """Display the grid"""
        out.clear_output(wait=True)
        with out:
            fig, ax = plt.subplots()
            # Hide the ticks
            ax.set_xticks([])
            ax.set_yticks([])
            # Display the grid
            ax.imshow(self.grid, cmap='Greens', vmin=0, vmax=2)
            plt.scatter(self.position, 0, s=1000)
            plt.show()

## Run the agent in Environment6

In [242]:
# Instanciate the agent in Environment6
interactions = [
    Interaction(0,0,-1),
    Interaction(0,1,1),
    Interaction(1,0,-1),
    Interaction(1,1,1)
]
a = Agent(interactions)
e = Environment6()

# Output widget for displaying the plot
out = Output()

# Run the interaction loop
step = 0
outcome = 0

Run the simulation step by step to see the Selection DataFrame. Use `Ctrl+Enter` to run the cell bellow and stay on it.

In [243]:
print(f"Step {step}")
step += 1
e.display()
display(out)
action = a.action(outcome)
outcome = e.outcome(action)
a.selection_df[['composite', 'weight', 'post_valence', 'action', 'proclivity', 'proclivity_sum', 'intended']]

Step 0


Output()

Action: 0, Prediction: 0, Outcome: 0, Prediction_correct: True, Valence: -1
Intended 00:-1


,composite,weight,post_valence,action,proclivity,proclivity_sum,intended
0,NaN,0.0,NaN,0.0,NaN,0.0,00
1,NaN,0.0,NaN,1.0,NaN,0.0,10


Figure 1 illustrates the learning mechanism and the interaction selection mechanism.

![Agent5](img/Figure_1_Agent7.svg)

Figure 1: Agent7 records and reinforces two levels of composite interactions:
* First-level composite interaction $c_{t-1} = (i_{t-2}, i_{t-1}: weight)$, 
* Second-level composite interaction $((i_{t-3}, i_{t-2}), i_{t-1}: weight)$, and $i_{t-3}, (i_{t-2}, i_{t-1}): weight)$. 

The last enacted primitive interaction $i_{t-1}$ and the last enacted composite interaction $c_{t-1}$ activates previously-learned composite interactions that propose their post interaction. 

Now the post interaction may be a composite interaction.

# ASSIGNMENT

In [236]:
class Agent7(Agent):
    def learn(self):
        # Implement the learning mechanism of Agent6 here
        pass

## Test your Agent7 in Environment6

In [239]:
# Instanciate a new agent
interactions = [
    Interaction(0,0,-1),
    Interaction(0,1,1),
    Interaction(1,0,-1),
    Interaction(1,1,1)
]
a = Agent7(interactions)
e = Environment6()

# Output widget for displaying the plot
out = Output()

# Run the interaction loop
step = 0
outcome = 0

In [ ]:
print(f"Step {step}")
step += 1
e.display()
display(out)
action = a.action(outcome)
outcome = e.outcome(action)
a.selection_df[['composite', 'weight', 'post_valence', 'action', 'proclivity', 'proclivity_sum', 'intended']]

## Test your Agent7 in the grid world

## Report 

Explain what you programmed and what results you observed. Export this document as PDF including your code, the traces you obtained, and your explanations below (no more than a few paragraphs):